<a href="https://colab.research.google.com/github/GeorgeTelles/detector_topos_fundos/blob/main/detector_topos_fundos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://postimg.cc/FYtnXpJw">
  <img src="https://i.postimg.cc/t470PmRR/logo-sem-fundo-01.png" alt="logo" width="300">
</a>

---
# **Detector de topos e fundos**

#### Este projeto em Python realiza a análise de dados financeiros para detectar automaticamente topos e fundos de um ativo. Utilizando técnicas de análise de séries temporais e detecção de padrões, o sistema identifica pontos críticos de máximo e mínimo no gráfico. Além disso, o projeto cria visualização dessas informações ao destacar os topos e fundos diretamente no gráfico, facilitando a interpretação e a tomada de decisões financeiras. A ferramenta é ideal para traders e investidores que buscam uma maneira automatizada de analisar tendências e padrões de mercado.


---

#1. Bibliotecas

In [ ]:
!pip install mplfinance

In [ ]:
import pandas as pd
import numpy as np
import pytz
import yfinance as yf
import mplfinance as mpf
import plotly.graph_objects as go

#2. Obtenção de dados OHLC

In [ ]:
df = yf.download('^BVSP', period="5y", interval='1d')

#3. Construção das Funções

In [ ]:
df_invertido = df.iloc[::-1]
df_invertido['Low'] <= df_invertido['Low'].rolling(2, closed='left').min()
pass

In [ ]:
def localizador_fundos(df, n_candles=5):
  df_invertido = df.iloc[::-1]

  cond_low_anteriores = df['Low'] <= df['Low'].rolling(n_candles, closed='left').min()
  cond_low_posteriores = df_invertido['Low'] <= df_invertido['Low'].rolling(n_candles, closed='left').min()

  return (cond_low_anteriores & cond_low_posteriores)

def localizador_topos(df, n_candles=2):
  df_invertido = df.iloc[::-1]

  cond_high_anteriores = df['High'] <= df['High'].rolling(n_candles, closed='left').max()
  cond_high_posteriores = df_invertido['High'] <= df_invertido['High'].rolling(n_candles, closed='left').max()

  return (cond_high_anteriores & cond_high_posteriores)

In [ ]:
localizador_fundos(df)
localizador_topos(df)
pass

In [ ]:
n_candles = 10
ls_vertices = ['neutro']

dict_topos_fundos = {'vertice': [], 'data': [], 'preco': []}

for i in range(len(df)):

  if localizador_fundos(df, n_candles).iloc[i]:

    if ls_vertices[-1] != 'fundo':
      dict_topos_fundos['vertice'].append('fundo')
      dict_topos_fundos['data'].append(df.index[i])
      dict_topos_fundos['preco'].append(df.Low.iloc[i])
      ls_vertices.append('fundo')

    else:
      if df.Low.iloc[i] < dict_topos_fundos['preco'][-1]:
        dict_topos_fundos['data'][-1] = df.index[i]
        dict_topos_fundos['preco'][-1] = df.Low.iloc[i]

  if localizador_topos(df, n_candles).iloc[i]:

    if ls_vertices[-1] != 'topo':
      dict_topos_fundos['vertice'].append('topo')
      dict_topos_fundos['data'].append(df.index[i])
      dict_topos_fundos['preco'].append(df.High.iloc[i])
      ls_vertices.append('topo')

    else:
      if df.High.iloc[i] > dict_topos_fundos['preco'][-1]:
        dict_topos_fundos['data'][-1] = df.index[i]
        dict_topos_fundos['preco'][-1] = df.High.iloc[i]

In [ ]:
df_topos_fundos = pd.DataFrame(dict_topos_fundos)
df_topos_fundos.index = df_topos_fundos.data

df_final = pd.merge(df, df_topos_fundos, left_index=True, right_index=True, how='left')

df_figure = df_final



df_topos_fundos.index = pd.to_datetime(df_topos_fundos.index)
df_final = pd.merge(df, df_topos_fundos, left_index=True, right_index=True, how='left')
df_figure = df_final

fig_topos_fundos = go.Figure(data=[go.Candlestick(
    name='',
    x=df_figure.index,
    open=df_figure.Open,
    high=df_figure.High,
    low=df_figure.Low,
    close=df_figure.Close,
    increasing_line_color='green',
    decreasing_line_color='red',
    showlegend=False)])

fig_topos_fundos.add_trace(go.Scatter(
    x=df_topos_fundos.index,
    y=df_topos_fundos.preco,
    mode='lines',
    line_width=2,
    line_dash='solid',
    line_color='black',
    name='Topos / Fundos',
    showlegend=True
))

fig_topos_fundos.update_xaxes(rangebreaks=[dict(bounds=['sat', 'mon'])])

fig_topos_fundos.update_layout(
    xaxis_rangeslider_visible=False,
    title_text='<b>Detector de Topos e Fundos',
    template='none',
    margin=dict(l=75, r=75, t=75, b=75),
    paper_bgcolor='#f7f8fa',
    width=1600,
    height=600
)

fig_topos_fundos.show()
